In [3]:
%matplotlib inline

In [4]:
import tensorflow as tf
import zipfile
import os
import pandas as pd
import numpy as np
import pathlib
import datetime
from collections import Counter
import copy
import matplotlib.pyplot as plt
import time

# Helpful Functions

In [6]:
def cleanURL(url):
    p = pathlib.Path(url)
    path = str(p.as_posix()) 
    return path 


def getDF(loc, sheetname):
    dataframe = pd.read_excel(loc, sheetname)
    #https://stackoverflow.com/questions/40950310/strip-trim-all-strings-of-a-dataframe
    dataframe = dataframe.applymap(lambda x: x.strip() if type(x) is str else x)
    return dataframe

def printNulls(df):
    null_columns = df.columns[df.isnull().any()]
    return df[null_columns].isnull().sum() 


def writeDFToFile(dfs, path_): #dfs is an array of dataframes and their sheet names , path needs to have
    time_ = str(datetime.datetime.now())
    current_date_time = time_[0:time_.index(".")]
    current_date_time = current_date_time.replace(":", "-")
    task4_fileoutput = path_+current_date_time+".xlsx"

    writer = pd.ExcelWriter(task4_fileoutput)
    
    for df_tuple in dfs:  
        df = df_tuple[0]
        sheetName = df_tuple[1]
        df.to_excel(writer, sheetName)
    print("file written to :       " + task4_fileoutput)
    writer.save()
    
def plotImg(img):
    if(type(img) ==type(None)):
        img =outputsDataReal[10000]
    arr = []
    for a in img:
        arr = [a] + arr
    plt.pcolor( arr, cmap = 'gist_ncar' )

    plt.show()    

cwd = os.getcwd()
cwd

'/home/muhammadayub/Desktop/CS230/Notebooks/re'

# Load Data

# Generate the Raw Data for the Neural Network 

In [7]:
CATEGORY_AMT = 34  # model5/model3_12_9__541_160.ckpt

In [8]:
devMiniBatches = np.load( '/home/muhammadayub/Desktop/CS230/models_saved/model5/devMiniBatches.npy')
testMiniBatches = np.load( '/home/muhammadayub/Desktop/CS230/models_saved/model5/testMiniBatches.npy' )
minibatches = np.load( '/home/muhammadayub/Desktop/CS230/models_saved/model5/minibatches.npy' )

In [9]:
devMiniBatches

array([360, 362, 316,  43, 246, 319, 293, 224,  41,  50, 274,  37,  65,
       266, 241, 311,  79,  18])

In [11]:
base_filepath = "/home/muhammadayub/Desktop/CS230/training_data/minibatches_real/"

a_ = time.time()

inputData = []
outputData = []

for i_ in range(374):
    
    inputPath = base_filepath+'input_'+str(i_)+"_.npy"
    outputPath = base_filepath+'output_'+str(i_)+"_.npy"
    
    inputMinibatch = np.load(inputPath, mmap_mode = 'r')
    outputMinibatch = np.load(outputPath, mmap_mode = 'r')
    
    oneHotVectors = np.zeros(shape=( len(outputMinibatch), CATEGORY_AMT ))
    
    
    for x_ , img in enumerate(outputMinibatch):
        cCount = Counter(img.flatten())
        mC = cCount.most_common()
        
        if(len(mC) == 2):
            if(mC[0][0] == -1):
                oneHotVectors[x_][int(mC[1][0])] = 1
            else:
                print("oh no-> assumption doesn't hold true . Recode")
        elif(len(mC) == 3):
            if(mC[0][0] == -1):
                oneHotVectors[x_][int(mC[1][0])] = 1#.5  # we are multihot encoding to make the values the same 
                oneHotVectors[x_][int(mC[2][0])] = 1#.5
            else:
                print("oh no-> assumption doesn't hold true . Recode")
        else:#(len(mc) >= 4):
            if(mC[0][0] == -1):
                oneHotVectors[x_][int(mC[1][0])] = 1#.33
                oneHotVectors[x_][int(mC[2][0])] = 1#.34
                oneHotVectors[x_][int(mC[3][0])] = 1#.33
            else:
                print("oh no-> assumption doesn't hold true . Recode")
        
            
            
    inputData.append(inputMinibatch)
    outputData.append(oneHotVectors)
    
    
b_ = time.time()
   
print(b_-a_)

483.55138969421387


In [12]:
print(len(inputData))
print(len(outputData))


374
374


In [29]:
outputData[0].shape

(200, 34)

# Get the model and learn from the data

In [31]:
def create_placeholders(n_H0, n_W0, n_C0, n_y):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_H0 -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes
        
    Returns:
    X -- placeholder for the data input, of shape [None, n_H0, n_W0, n_C0] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """


    X = tf.placeholder(shape =[None, n_H0, n_W0, n_C0], dtype = np.float32, name="X")
    Y = tf.placeholder(shape  =[None, n_y], dtype = np.float32 , name="Y")
    ### END CODE HERE ###
    
    return X, Y

In [32]:
def initialize_parameters():    

    W1 = tf.get_variable("W1", [10, 10, 28, 3], initializer =tf.contrib.layers.xavier_initializer(seed = 0))
    W2 = tf.get_variable("W2", [6, 6, 3, 2], initializer =tf.contrib.layers.xavier_initializer(seed = 0))
    W3 = tf.get_variable("W3", [35,200] , initializer =tf.contrib.layers.xavier_initializer(seed = 0) )

    parameters = {"W1": W1, "W2": W2 , 'W3': 'W3'}
    
    return parameters

In [33]:
def forward_prop(X, params):
    W1 = params['W1']
    W2 = params['W2']    
    W3 = params['W3']
    
    #convolution 
    Z1 = tf.nn.conv2d(X,W1, strides = [1,2,2,1], padding = 'VALID')
    
    #bias added automatically # RELU
    A1 = tf.nn.relu(Z1)
    
    #average pooling -> at this point all features/weights are important to us
    P1 = tf.nn.avg_pool(A1, ksize = [1,3,3,1], strides = [1,1,1,1], padding = 'SAME')

    # convolution 
    Z2 = tf.nn.conv2d(P1,W2, strides = [1,2,2,1], padding = 'VALID')
    
    #RELU
    A2 = tf.nn.relu(Z2)
    
    #max pooling
    P2 = tf.nn.max_pool(A2, ksize = [1,3,3,1], strides = [1,1,1,1], padding = 'VALID')
    
    #flatten
    P2 = tf.contrib.layers.flatten(P2)

    #fully connected
    Z3 = tf.contrib.layers.fully_connected(P2, CATEGORY_AMT, activation_fn = None) #1 for yes/no
    #going to add the softmax directly

    return Z3

In [24]:
graph = tf.Graph()
with graph.as_default():
    with tf.Session(graph=graph) as sess:
        ab = tf.constant(np.array([[1,2,4],[42,1,5]]), dtype = np.float32)
        bc = tf.constant(np.array([[5,3,4],[42,1,6]]), dtype = np.float32)
        
        equals = tf.equal(ab, bc)
        equalsFloat32 = tf.cast(equals, tf.float32)
        
        sumVal = tf.reduce_sum(equalsFloat32, axis = 1)
        sumValDivided = sumVal/3.0
        
        meanReduced = tf.reduce_mean (sumValDivided )
        
        result = sess.run([meanReduced])
        print(result[0])
        
        
        
        x = tf.constant([[1,2],[3.,4]])
        
        mean = tf.reduce_mean(x, axis = 0)
        print(sess.run([mean]))
        
        

0.5
[array([2., 3.], dtype=float32)]


In [78]:
# 22:04 - 22:08 Tuesday
# 22:10 - 22:20
#Restoring the model


In [116]:
filepath = "/home/muhammadayub/Desktop/CS230/models_saved/model5/model3_12_9__829_100.ckpt" #4/model3_12_9__134_100.ckpt" #model4/model3_12_9__9_80.ckpt"
# dev_test_or_train = 1
# if(dev_test_or_train ==1): #train

# filepathEvalMetrics = "/home/muhammadayub/Desktop/CS230/models_saved/model3/model_12_9__"+str(49)+"_"+str(60)+"_evalMetrics_train.npy"
accuracies= []

print(" Running ", len(outputData) , " minibatches to get the evaluation metrics")
output_classes = []
predicted_classes = []

preds_one_hot_ev_value = None
output_one_hot_ev_value =None 


graph = tf.Graph()
with graph.as_default(): 
    with tf.Session(graph=graph) as sess:
        
        #define the graph
        X, Y = create_placeholders(64, 64, 28, 34)
        parameters = initialize_parameters()
        Z3 = forward_prop(X, parameters)
        #optimization 
#         cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y))
#         optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)#1e-4).minimize(cross_entropy)

        #calculating the accuracy of the model 
        softmaxZ3 = tf.nn.softmax(Z3) # size will be 2200, 34
        
        top4 =  tf.nn.top_k(softmaxZ3, 3)     
        top4Indices = top4.indices
        oneHotIndices = tf.one_hot(top4Indices, depth = 34)
        summationOfOneHotFor_NHotEncoding = tf.reduce_sum(oneHotIndices, 1)  # this is the N Hot Encoding 

        
        #equals = tf.equal(summationOfOneHotFor_NHotEncoding, Y) #boolean tensor
        #equalsFloat32 = tf.cast(equals, tf.float32)  # 1 or 0 tensor
        
        summationOfOneHotFor_NHotEncoding = tf.cast(summationOfOneHotFor_NHotEncoding, tf.int32)
        Y = tf.cast(Y, tf.int32)
        
        andOp = tf.bitwise.bitwise_and(summationOfOneHotFor_NHotEncoding, Y)
        andOp = tf.cast(andOp, tf.float32)
        sumVal = tf.reduce_sum(andOp, axis = 1)
        sumValDivided = sumVal/3.0
        
        
        accuracy = tf.reduce_mean(sumValDivided)
        
        
        saver = tf.train.Saver()

        saver.restore(sess , filepath)
        print("Model restored.")
        
        #devMiniBatches 
        #testMiniBatches
        for i_  in range(len(outputData)):

            output_image64 = outputData[i_]
            inputImage64 = inputData[i_]
            
            
            
            summationOfOneHotFor_NHotEncoding_val, Y_val, andOpVal, accuracy_val  = sess.run([summationOfOneHotFor_NHotEncoding,Y,andOp, accuracy], feed_dict={X: inputImage64, Y: output_image64})
#             equalsFloat32_val = sess.run([equalsFloat32], feed_dict={X: inputImage64, Y: output_image64})
            
            
#             sumValDivided, accuracy_val, nHotEncoding = sess.run([sumValDivided, accuracy, summationOfOneHotFor_NHotEncoding], feed_dict={X: inputImage64, Y: output_image64})
#             sumValDivided_val,accuracy_val,nHotEncoding = sess.run([sumValDivided, accuracy, summationOfOneHotFor_NHotEncoding], feed_dict={X: inputImage64, Y: output_image64})
            
            accuracies.append(accuracy_val)
            output_classes.append(Y_val)
            predicted_classes.append(summationOfOneHotFor_NHotEncoding_val)
            
            print('Minibatch: ', str(i_), " ", datetime.datetime.now())
            


 Running  374  minibatches to get the evaluation metrics
INFO:tensorflow:Restoring parameters from /home/muhammadayub/Desktop/CS230/models_saved/model5/model3_12_9__829_100.ckpt
Model restored.
Minibatch:  0   2018-12-12 11:07:56.937919
Minibatch:  1   2018-12-12 11:07:57.288890
Minibatch:  2   2018-12-12 11:07:57.575529
Minibatch:  3   2018-12-12 11:07:57.844528
Minibatch:  4   2018-12-12 11:07:58.212810
Minibatch:  5   2018-12-12 11:07:58.491627
Minibatch:  6   2018-12-12 11:07:58.775402
Minibatch:  7   2018-12-12 11:07:59.116766
Minibatch:  8   2018-12-12 11:07:59.403766
Minibatch:  9   2018-12-12 11:07:59.676215
Minibatch:  10   2018-12-12 11:08:00.022593
Minibatch:  11   2018-12-12 11:08:00.304302
Minibatch:  12   2018-12-12 11:08:00.567827
Minibatch:  13   2018-12-12 11:08:00.864334
Minibatch:  14   2018-12-12 11:08:01.160819
Minibatch:  15   2018-12-12 11:08:01.423525
Minibatch:  16   2018-12-12 11:08:01.691091
Minibatch:  17   2018-12-12 11:08:02.026356
Minibatch:  18   2018-12

Minibatch:  181   2018-12-12 11:08:50.748082
Minibatch:  182   2018-12-12 11:08:51.081066
Minibatch:  183   2018-12-12 11:08:51.355018
Minibatch:  184   2018-12-12 11:08:51.616716
Minibatch:  185   2018-12-12 11:08:51.959683
Minibatch:  186   2018-12-12 11:08:52.257072
Minibatch:  187   2018-12-12 11:08:52.533107
Minibatch:  188   2018-12-12 11:08:52.883347
Minibatch:  189   2018-12-12 11:08:53.174185
Minibatch:  190   2018-12-12 11:08:53.459052
Minibatch:  191   2018-12-12 11:08:53.812061
Minibatch:  192   2018-12-12 11:08:54.073824
Minibatch:  193   2018-12-12 11:08:54.350531
Minibatch:  194   2018-12-12 11:08:54.694457
Minibatch:  195   2018-12-12 11:08:54.965388
Minibatch:  196   2018-12-12 11:08:55.234366
Minibatch:  197   2018-12-12 11:08:55.583514
Minibatch:  198   2018-12-12 11:08:55.896066
Minibatch:  199   2018-12-12 11:08:56.174607
Minibatch:  200   2018-12-12 11:08:56.416125
Minibatch:  201   2018-12-12 11:08:56.659244
Minibatch:  202   2018-12-12 11:08:56.985371
Minibatch:

Minibatch:  364   2018-12-12 11:09:44.511959
Minibatch:  365   2018-12-12 11:09:44.786350
Minibatch:  366   2018-12-12 11:09:45.128769
Minibatch:  367   2018-12-12 11:09:45.420694
Minibatch:  368   2018-12-12 11:09:45.694279
Minibatch:  369   2018-12-12 11:09:46.026151
Minibatch:  370   2018-12-12 11:09:46.299485
Minibatch:  371   2018-12-12 11:09:46.562138
Minibatch:  372   2018-12-12 11:09:46.907765
Minibatch:  373   2018-12-12 11:09:47.169442


In [119]:
# print(summationOfOneHotFor_NHotEncoding_val[0])
print(Y_val[0])
# print(andOpVal[0])
# type(andOpVal)

# andOpVal.shape
# summationOfOneHotFor_NHotEncoding_val.shape
len(predicted_classes)

[0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]


374

In [120]:
            
#new graph for concatenating things and getting into the right format for the F1 score 
graph = tf.Graph()
with graph.as_default(): 
    with tf.Session(graph=graph) as sess:            
    
        val1 = tf.concat(output_classes, axis = 0)
        
    
        #after running the minibatches-> we can concatenate the values 
        preds = tf.concat(predicted_classes,axis = 0)
        data_outputs = tf.concat(output_classes,axis = 0)
        
        preds_one_hot_ev, output_one_hot_ev = sess.run([preds, data_outputs])
        preds_one_hot_ev_value=preds_one_hot_ev
        output_one_hot_ev_value =output_one_hot_ev
        
#         d = 35 # d is the total number of classes you have 
        #classLabels =  np.array([2,3,5,6,5,4,2,0,0])# the indexes 
        # tf.one_hot(classLabels, d)

#         preds_one_hot = tf.one_hot(preds, d)
#         data_outputs_one_hot =tf.one_hot(data_outputs, d)#evaluated 
#         preds_one_hot_ev, output_one_hot_ev = sess.run([preds_one_hot, data_outputs_one_hot])
        
#         preds_one_hot_ev_value =preds_one_hot_ev
#         output_one_hot_ev_value =output_one_hot_ev
        
#For Debugging below:
#         zVal = sess.run([Z3], feed_dict={X: inputImage64, Y: output_image64})
#         softMaxVal = sess.run([softmaxZ3], feed_dict={X: inputImage64, Y: output_image64})
#         output_class_val = sess.run([output_class], feed_dict={X: inputImage64, Y: output_image64})
#         num_correct_val = sess.run([num_correct], feed_dict={X: inputImage64, Y: output_image64})
#         num_correct_to_int_val = sess.run([num_correct_to_int], feed_dict={X: inputImage64, Y: output_image64})

In [121]:
_y_true_ = None
_y_pred_ = None 
_f1Score_ = None 
_precision_ = None 
_recall_ =None 
TPVal = None
FPVal = None
FNVal = None 
_f1_ = None 
#using the F1 function found online -> get the values for the predictions  
graph = tf.Graph()
with graph.as_default(): 
    y_true = tf.Variable(output_one_hot_ev_value)
    y_pred = tf.Variable(preds_one_hot_ev_value)
    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)

    #moded from -> https://stackoverflow.com/questions/35365007/tensorflow-precision-recall-f1-score-and-confusion-matrix
    TP = tf.count_nonzero(y_pred * y_true, axis=0) 
    FP = tf.count_nonzero(y_pred * (y_true - 1), axis=0) 
    FN = tf.count_nonzero((y_pred - 1) * y_true, axis=0)  # .001 for numerical stability 

    #they are integers right now and should be floats for numerical stability 
    TP = tf.cast(TP, tf.float32)
    FP = tf.cast(FP, tf.float32)
    FN = tf.cast(FN, tf.float32)
    
    
    precision = TP / (TP + FP+tf.constant(.001))
    recall = TP / (TP + FN+tf.constant(.001))
    f1 = 2 * precision * recall / (precision + recall+tf.constant(.001))

    f1Score = tf.reduce_mean(f1)

    
    with tf.Session(graph=graph) as sess:    
        tf.global_variables_initializer().run(session=sess)
        _f1_, _y_true_, _y_pred_, _f1Score_ , _precision_ ,_recall_ , TPVal,FPVal, FNVal = sess.run([f1, y_true, y_pred, f1Score, precision ,recall, TP,FP, FN ])          


In [122]:
sum(np.array(accuracies).flatten())/len(accuracies)*100

74.99286914094884

In [126]:
_f1_
# _precision_
# _recall_ 
_f1Score_

array([0.        , 0.        , 0.91166234, 0.        , 0.        ,
       0.        , 0.64018273, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.9685871 , 0.        ], dtype=float32)

In [117]:
data = np.array([[0. ,  0.34, 0.33 ,0. ,  0.  , 0. ,  0. ,  0.   ,0.  , 0. , 
                     0.   ,0. ,  0.,   0., 0.  , 0. ,  0. ,  0.,  0.   ,0.  , 0.  ,
                     0. ,  0. ,  0. ,  0.  , 0. ,  0.  , 0., 0. ,  0. ,  0. ,
                     0.  , 0.33, 0.  ], 
                [0.27 ,  0., 0. ,0.35 ,  0.38  , 0. ,  0. ,  0.   ,0.  , 0. , 
                     0.   ,0. ,  0.,   0., 0.  , 0. ,  0. ,  0.,  0.   ,0.  , 0.  ,
                     0. ,  0. ,  0. ,  0.  , 0. ,  0.  , 0., 0. ,  0. ,  0. ,
                     0.  , 0., 0.  ]])

graph = tf.Graph()
with graph.as_default(): 
    with tf.Session(graph=graph) as sess:
        x = tf.placeholder(shape = (2,34), dtype=np.float32)
        top4 =  tf.nn.top_k(x, 3)     

        top4Indices = top4.indices
        oneHotIndices = tf.one_hot(top4Indices, depth = 34)
        summationOfOneHotFor_NHotEncoding = tf.reduce_sum(oneHotIndices, 1)
        
#         top4Vals = sess.run([top4], feed_dict={x:data})
# top4Vals[0].indices
        top4IndicesVals = sess.run([top4Indices], feed_dict={x:data})
        
        oneHotIndicesValues = sess.run([oneHotIndices], feed_dict={x:data})
        
        NHotEncodingValues = sess.run([summationOfOneHotFor_NHotEncoding], feed_dict={x:data})
        
        print(oneHotIndicesValues[0].shape)
        print(top4IndicesVals[0].shape)
        
        print(oneHotIndicesValues)
        print(top4IndicesVals)
        
        print('----------------')
        
        print(NHotEncodingValues[0].shape)
        
        print(NHotEncodingValues)
        
        
        

# tf.math.top_k 

(2, 3, 34)
(2, 3)
[array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0.]],

       [[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 